In [2]:
import numpy as np
import pandas as pd
def word_tokenizer(input : str, n_unique : int=None):
  '''
  Converts input string into an array of word tokens, splitting at any whitespace.
  Parameters:
    input (str): input with whitespace
    n_unique (int): number of unique tokens. The most common n-1 tokens in the original vocabulary are kept;
    all tokens outside this vocabulary are cast to <unk>
  Returns:
    tokens (np.ndarray of str)
  '''
  tokens = np.array(input.split())
  unique_tokens = np.unique(tokens, return_counts=True)
  if n_unique is not None:
    unique_tokens_freq_sort = unique_tokens[0][np.argsort(-unique_tokens[1])]
    unk_tokens = unique_tokens_freq_sort[n_unique-1:len(unique_tokens_freq_sort)]
    tokens[np.isin(tokens, unk_tokens)] = '<unk>'
  return np.array(tokens)
class MarkovChain:
  '''
  MarkovChain - can be trained to get transition probabilities, can be evaluated to get next n words.
  The format is kind of silly for markov chains, but should be something which can be
  relatively unmodified as the model complexity and training sample size grow
  '''
  def __init__(self):
    return
  def train(self, X : np.ndarray, pseudocount : int=0):
    '''
    Trains markov chain (populates transition probability matrix)
    Parameters:
      X (np.ndarray): tokens. This should already be of a reasonable vocabulary size 
        (i.e. you should replace extra words with <unk> in the tokenizer!)
      pseudocount (int): amount to add to each element of count matrix (as a form of regularization) 
    '''
    unique = np.unique(X)
    # I honestly am not sure if there's a way to do this which relies more on built-in numpy functions
    # but its big-O runtime is constrained by O(v^2 + n) anyways, which I don't think we can improve upon.
    token_to_idx = dict()
    for i in range(len(unique)):
      word = unique[i]
      token_to_idx[word] = i
    # we could use a sparse matrix here but that only works if we have 0 for pseudocount; probably fine either way.
    # It's a toy model!
    transition_matrix = np.zeros([len(unique), len(unique)])
    prev_token = None
    for token in X:
      if prev_token is not None:
        prev_idx = token_to_idx[prev_token]
        cur_idx = token_to_idx[token]
        transition_matrix[prev_idx, cur_idx] += 1
      prev_token = token
    for i in range(len(unique)):
      transition_matrix[i] /= np.sum(transition_matrix[i])
    self.transition_matrix = transition_matrix
    self.token_to_idx = token_to_idx
  def predict(self, X : np.ndarray, n : int=1):
    '''
    Continues input token array for next n tokens (cannot specify empty X)
    Parameters:
      X (np.ndarray): tokens to continue
      n (int): number of tokens to output
    Returns:
      continuation (np.ndarray): tokens to output
    '''
    idx_to_token = {idx: token for token, idx in self.token_to_idx.items()}
    token_to_idx = self.token_to_idx
    prev_idx = token_to_idx[X[-1]]
    res_tokens = []
    for i in range(n):
      transition_probs = self.transition_matrix[prev_idx]
      next_idx = np.random.choice(len(token_to_idx), size=1, p=transition_probs)[0]
      res_tokens.append(idx_to_token[next_idx])
      prev_idx = next_idx
    return np.array(res_tokens)
  # TODO maybe worth adding a get_log_likelihood thing here to compare loss with other models

In [3]:
testStr = 'a a a b b b a a b b b b b b a a a a b b b c c a c c c a b b b c a a a c a a a a a a a a a a a'
tokens = word_tokenizer(testStr)
model = MarkovChain()
model.train(tokens)
model.predict(tokens, 10)

array(['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'c', 'c'], dtype='<U1')

In [4]:
from zipfile import ZipFile
  
# downloaded from https://deepai.org/dataset/text8
# upon the recommendation of a reddit post saying it's a good dataset with a small vocab size
# which is good for model testing while we're still using word-level tokenization
file_name = "text8.zip"
with ZipFile(file_name, 'r') as zip:
  text8_raw = str(zip.read('text8'))

In [25]:
print(text8_raw[0:1000])
# 4MB, seems reasonable
text8_raw_small = text8_raw[0:4000000]
tokens = word_tokenizer(text8_raw_small, n_unique=5000)

b' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic institu

In [26]:
unique_tokens = np.unique(tokens, return_counts=True)
unique_tokens_freq_sort = unique_tokens[0][np.argsort(-unique_tokens[1])]
print(unique_tokens_freq_sort)
unique_tokens_counts = unique_tokens[1][np.argsort(-unique_tokens[1])]
print(unique_tokens_counts)

['<unk>' 'the' 'of' ... 'mystic' 'celestial' 'sole']
[91882 42015 24097 ...    13    13    13]


Unlucky. Even with 5000 unique words, there are twice as many unknown words as 'the's, and we are beginning to hit the limits of what is feasible under this implementation in terms of memory usage.

In [31]:
model = MarkovChain()
model.train(tokens)
# apparently np.random.seed is bad for use in specific classes,
# but it's probably fine for a notebook setting.
np.random.seed(1234)
model.predict(np.array(['the']), 100)

array(['<unk>', 'might', 'be', 'seen', 'much', 'improved', 'but', 'the',
       'village', 'see', 'also', 'has', 'low', 'point', 'in',
       'particular', 'individuals', '<unk>', 'rival', 'newspapers', 'are',
       'normally', '<unk>', 'by', 'their', 'name', 'for', 'the',
       'central', 'district', 'the', 'faithful', 'saints', '<unk>', 'one',
       'one', 'four', 'zero', 's', 'there', 'are', 'against', '<unk>',
       'of', 'property', 'is', '<unk>', 'inspired', 'by', 'jews', 'and',
       'moderate', 'republican', '<unk>', 'may', 'therefore', 'stephen',
       'v', 'v', 'pp', 'four', 'seven', 'one', 'five', 'little', '<unk>',
       'football', 'while', '<unk>', '<unk>', 'parts', 'of', 'it', 'as',
       'a', 'tax', 'bases', 'its', '<unk>', 'animation', 'film', 'making',
       'them', 'other', 'pop', 'art', 'opened', 'the', 'murder', 'is',
       'a', 'means', 'of', 'time', 'largely', 'composed', 'of', 'the',
       '<unk>', 'of'], dtype='<U11')

Oof. At the very least, we see some nice two-word patterns, and even three-word patterns as a result of chained two-word associations. For example, we see 'might be' and 'be seen' chained into 'might be seen', which is a coherent three-word phrase! Still, it seems to have no awareness of where it is in a sentence. To be honest, more sentence-level awareness might be a bit difficult to obtain with the text8 dataset, even with a more advanced model, as it does not have punctuation. It's probably better to apply methods like byte-pair encoding before incorporating more complex data so as to avoid wasting time hard-coding a bunch of data cleaning heuristics.
